D-LUT Demo

In [ ]:
# Load necessary modules and set working directory
%load_ext autoreload
%autoreload 2

import torch
# device = torch.device("cuda:2")
import os
import argparse
import yaml
import codebase as cb
from codebase.Exp2Runner import Exp2Runner


In [18]:
# Configuration settings
config_string = """
## Basic config
seed: 12213
root_dir: ./Log

## Model settings
model_type: "Energy"
swish: "True"

## Training settings
train_epochs: 15
save_epoch: ~
save_model_flag: True
batch_size: 128
learning_rate: 0.001
sigma: 0.005

## Data Generation settings
style_dir: ./Imgs/style/img0.jpg
style_shape: 
- 400
- 300
add_uniform: False
uniform_percentage: 0.05

## Langevin settings
L_steps: 40
eps: 0.0002

## Content settings
content_dir: ./Imgs/content
content_shape:
- 300
- 400

## LUT settings
lut_dim: 15
## N 
lut_sample: 50

## Eval and Visualization settings
model_dir: ~
vis_dim: 20
"""

args = yaml.load(config_string,Loader=yaml.FullLoader)
args=  cb.utils.dict2namespace(args)

In [ ]:
# Training
runner = Exp2Runner(args)
runner.train()
print(f"Finish Training")

In [ ]:
# Eval
args.model_dir = './Log/models/Img0/model_15_0.005.pth'
eval_runner = Exp2Runner(args)
eval_runner.device = device
eval_runner.eval()
# "This code will generate three types of images: direct Langevin sampling on the image, Langevin sampling on the LUT, and color transfer using the sampled LUT."
eval_runner.visualization()
eval_runner.save_config()
# Use "tensorboard --logdir=./Log" to view result

In [ ]:
# Other Visualization functions
eval_runner.show_style()
eval_runner.show_score()

In [ ]:
# Play with LUTs 
from codebase.LUT import save_3dlut_to_file, draw_3d
import matplotlib.pyplot as plt
lut_list = eval_runner.lut_langevie()


for i in range(len(lut_list)):
    lut = lut_list[i] # lut [B,dim,dim,dim,3]
    mean_lut = lut.mean(dim=0) #[B,dim,dim,dim,3] > [dim,dim,dim,3]
    vis_lut = mean_lut.permute(3,0,1,2) # > [3,dim,dim,dim]

    lut_name = f"LUT_{i}" 
    #save LUTs to .CUBE
    save_3dlut_to_file(vis_lut,"./LUT",lut_name)

    #View LUTs
    fig = plt.figure()
    ax = plt.subplot(111, projection='3d')
    draw_3d(vis_lut,ax=ax)


In [ ]:
# Style Transfer
content_list = ['./Imgs/content']
savepath ="./Imgs/result"
lut_list = [lut.to(device) for lut in lut_list]
eval_runner.vis_and_save(savepath,content_list,lut_list,content_shape=[-1,-1],sample_step=5)